### 初始化

In [1]:
import os
import re
from JoinAgent import *

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')
project_root = os.path.dirname(dotenv_path)
file_path="final_markdown.md"

# 初始化API
llm=MultiLLM()

# 采用Divider类方法切割文本
divider=TextDivider(threshold=4096,overlap=128)

parser=LLMParser()

### 01 提取数学对象

In [2]:
data_template01='''
{"pos1":['数学对象1','数学对象2',...]}
'''
prompt_template01 ='''
你是一个工作细致的助手。我将给你一段数学类资料上的文本，请你帮我抽取出文本中所有的实体数学对象，并统一放入一个列表。
在工作期间，你将全程关闭搜索功能以及与外部的连接，仅凭文本本身内容来完成这项工作，不要擅自添加新的数学对象。
如果你提取出了一个数学对象，但你无法在文本中找到它的定义，请你不要输出这个数学对象。
请你除了输出这个列表外，不要在你的输出开头和结尾添加其他的东西。
提取的格式是：
{data_template}

特别注意：请你遇到文本中的示例，例题，习题等题目时直接跳过，不要解析其中的内容！！
数字、数学算式、代数式、字母等不含汉字的内容不被视作数学对象，请你删去！
请你不要输出类似于"函数f"，"矩阵B"这样并没有定义普适性、只是上下文中定义的指代性数学对象。

示例：
Input:本书中的函数都是在实数范畴内变化的.这一章我们首先介绍有关实数集
的基本概念和一些性质;继而讨论函数的概念及其一般性态,同时对基本初等函
数的性质概括地做一些回顾.本章的内容是初等数学某些知识的复习、总结和提
高,也是学习微积分的基础.

Output:{{"pos1":['函数','实数集','基本初等函数','微积分']}}
以上是示例，请你不要在正文中输出
############################################
以下是我给你的文本：{pos1}，请你帮我提取出数学对象，并放入一个列表。
'''

def validation01(text):
    return True

correction_prompt= '''
    你是一个严谨的校对员。我将给你一个由大模型生成的数据结构，请你根据规定格式内容进行校对和修正。

    校对的格式是：
    {data_template}

    以下是待校验的文本：{answer}，请你帮我校对和修正这个列表。
    '''

empty_template01='''
[]
'''

### 处理数学对象列表

In [3]:
def contains_chinese(text):
    return re.search(r'[\u4e00-\u9fff]', text) is not None

def filter_dict_of_lists(data):
    filtered_data = {}
    for key, value in data.items():
        filtered_pos1 = [item for item in value['pos1'] if contains_chinese(item)]
        filtered_data[key] = {'pos1': filtered_pos1}
    return filtered_data

def process_dict_of_lists(data):
    flat_list = [item for sublist in data.values() for item in sublist['pos1']]
    embeddings_dict = llm.embed_list(flat_list)
    similarity_dict = llm.partition_by_similarity(embeddings_dict, threshold=0.9)

    name_mapping = {}
    for key in flat_list:
        if key in similarity_dict:
            if similarity_dict[key]['Similar_keys']:
                name_mapping[key] = similarity_dict[key]['Similar_keys'][0]
            else:
                name_mapping[key] = key
        else:
            name_mapping[key] = key

    new_data = {}
    for key, value in data.items():
        new_pos1 = [name_mapping[item] for item in value['pos1']]
        new_data[key] = {'pos1': list(set(new_pos1))}  # 使用 set 去重

    return new_data

In [4]:
def create_entity_pairs(processed_data):
    entity_pairs = {}
    new_index = 1

    for index, data in processed_data.items():
        entities = data['pos1']
        n = len(entities)
        
        for i in range(n):
            for j in range(i+1, n):
                entity1 = entities[i]
                entity2 = entities[j]
                
                # 确保实体对的顺序一致
                if entity1 > entity2:
                    entity1, entity2 = entity2, entity1

                pair = (entity1, entity2)
                
                if pair in entity_pairs:
                    # 如果实体对已存在，更新共同出现的索引列表
                    entity_pairs[pair]['pos3'].append(index)
                else:
                    # 如果是新的实体对，创建新条目
                    entity_pairs[pair] = {
                        'pos1': entity1,
                        'pos2': entity2,
                        'pos3': [index]
                    }

    # 将结果转换为所需的格式
    result = {}
    for pair, data in entity_pairs.items():
        result[new_index] = data
        new_index += 1

    return result

In [5]:
def extract_entities(processed_data):
    entity_dict = {}
    new_index = 1

    for original_index, data in processed_data.items():
        entities = data['pos1']
        
        for entity in entities:
            if entity in entity_dict:
                # 如果实体已存在，更新它出现的索引列表
                entity_dict[entity]['pos2'].append(original_index)
            else:
                # 如果是新的实体，创建新条目
                entity_dict[entity] = {
                    'pos1': entity,
                    'pos2': [original_index]
                }

    # 将结果转换为所需的格式
    result = {}
    for entity, data in entity_dict.items():
        result[new_index] = data
        new_index += 1

    return result


In [6]:
def replace_indices_with_text(data_dict, text_list):
    result = {}
    
    for new_index, item in data_dict.items():
        # 复制原始项
        new_item = item.copy()
        
        # 动态检查哪个键对应着列表
        index_key = None
        for key, value in item.items():
            if isinstance(value, list) and all(isinstance(i, int) for i in value):
                index_key = key
                break
        
        # 如果没有找到对应列表的键，跳过这一项
        if index_key is None:
            result[new_index] = new_item
            continue
        
        # 获取索引列表
        indices = item[index_key]
        
        # 获取对应的文本段落并聚合
        texts = [text_list[i-1] for i in indices if 1 <= i <= len(text_list)]  # 确保索引在有效范围内
        aggregated_text = ' '.join(texts)
        
        # 用聚合的文本替换索引列表
        new_item[index_key] = aggregated_text
        
        # 将新项添加到结果字典
        result[new_index] = new_item
    
    return result


### 02 提取数学对象属性

In [7]:
data_template02='''
    {
    "数学对象":"数学对象名称",
    "定义":"...",
    "性质":["...",...],
    "适用条件":"...",
    "应用场景":"...",
    ...
        }
'''
prompt_template02 ='''
    这里有一段文本:{pos2}.
    我想要研究其中概念{pos1}的定义、性质、应用等属性，你来负责帮我从文本中帮我抽取相关内容，并按给定格式输出。
    这里有很多数学概念，但是我只要{pos1}这一个概念的解释
    提取的格式是一个单独的字典：
    {data_template}
    

    示例：
        {{
            "数学对象": "矩阵",
            "定义": "一个由有序数组组成的矩形阵列",
            "性质": ["可加性"],
            "计算方法"："矩阵运算方法",          
            ...
        }}

    以上是示例，请你不要在正文中输出
    ############################################
    字典中，第一个键是"数学对象"，值必须是{pos1}，即列表中字典的个数和所给数学对象的个数应相同。
    注意：字典中，键“性质”所对应的值都要是一个列表，列表内元素都要是“...性”的形式，如“对称性”，“连续性”等。如果无法概括为这种形式，则不输出这个键值对。
    字典中只有数学对象是必要的，如定义，性质，应用场景等键，如果在文本中找不到对应的值，则直接删除这个键。
    每个键的值不要超过50字。

    如果你觉得有其他键可以描述数学对象的属性，也可以添加，但注意：
    1.不要有"子类"，"相关联的数学对象"这种包含数学对象间关系的键出现；
    2.不要有跟已规定的键相似度较高的键，如"属性"键和规定的"性质"键内容相似度高，则不要输出这样的键；
    3.如果添加新键，要保证该键名称书面化，规范，简洁，且与其他键的值没有较高关联性。

    ""中不能出现字母，但如果字母未在该行第一个""中出现，则保留。
    不要解析文本中所有的示例。
    如果""中含顿号，则拆开成两个""分别储存内容。
    ""中如果为数学对象或一句句子，则不做修改；如果是一个状语或不完整的句子，则删减为数学对象，但前提是保证修改内容最少。
    请你除了输出数学对象和它们的性质之外，不要在你的回答最前面或最后面输入解释的语句。
    '''

def validation02(text):
    return True

correction_prompt= '''
    你是一个严谨的校对员。我将给你一个由大模型生成的数据结构，请你根据规定格式内容进行校对和修正。

    校对的格式是：
    {data_template}

    以下是待校验的文本：{answer}，请你帮我校对和修正这个列表。
    '''

empty_template02='''
[]
'''

### 合并属性字典

In [8]:
def merge_results(results):
    merged_dict = {}
    for item in results:
        key = item['数学对象']  # 获取数学对象名称
        if key in merged_dict:
            for k, v in item.items():
                if k == '性质':
                    if k in merged_dict[key]:
                        merged_dict[key][k] = list(set(merged_dict[key][k] + v))  # 性质存储在列表中，合并并去重
                    else:
                        merged_dict[key][k] = v
                elif k != '数学对象':
                    if k in merged_dict[key]:
                        merged_dict[key][k] += f";{v}"  # 不是性质的项，用分号连接
                    else:
                        merged_dict[key][k] = v
        else:
            merged_dict[key] = item
    return list(merged_dict.values())  # 返回合并后的字典值列表


### 03 提取数学对象关系

In [9]:
data_template03='''
    {
    "出发节点":"数学对象1",
    "到达节点":"数学对象2",
    "关系名称":"具体名称",
    "关系解释"："根据具体文本内容确定",
    "关系强度":用1-10间的整数进行评分
    }
'''
prompt_template03left ='''
    这里有一段文本:{pos3}
    你是一个工作细致的助手，负责帮我解释{pos1}和{pos2}之间的关系。
    '关系名称'的值的格式总应该是'...关系'，包括但不限于包含关系，特化关系，并列关系...如果无法概括成这种形式，也尽量用书面化语言表述。
    特别注意的是，关系是有向的，根据出发节点和到达节点的不同而不同。
    提取的格式是：{data_template}

    
    示例：
    {{
    "出发节点":"函数",
    "到达节点":"偶函数",
    "关系名称":"特化关系",
    "关系解释":"若函数f(x)满足对于任何x，都有f(x)=f(-x)，则称f(x)为偶函数"
    "关系强度":"10"
    }}

    特别注意：出发节点是{pos1}，到达节点是{pos2}
    ""中不能出现字母，但如果字母未在该行第一个""中出现，则保留。
    不要解析文本中所有的示例。。
    '''

prompt_template03right ='''
    这里有一段文本:{pos3}
    你是一个工作细致的助手，负责帮我解释{pos2}和{pos1}之间的关系。
    '关系名称'的值的格式总应该是'...关系'，包括但不限于包含关系，特化关系，并列关系...如果无法概括成这种形式，也尽量用书面化语言表述。
    特别注意的是，关系是有向的，根据出发节点和到达节点的不同而不同。
    提取的格式是：{data_template}

    
    示例：
    {{
    "出发节点":"函数",
    "到达节点":"偶函数",
    "关系名称":"特化关系",
    "关系解释":"若函数f(x)满足对于任何x，都有f(x)=f(-x)，则称f(x)为偶函数"
    "关系强度":"10"
    }}

    特别注意：出发节点是{pos2}，到达节点是{pos1}
    ""中不能出现字母，但如果字母未在该行第一个""中出现，则保留。
    不要解析文本中所有的示例。。
    '''

def validation03(text):
    return True

correction_prompt= '''
    你是一个严谨的校对员。我将给你一个由大模型生成的数据结构，请你根据规定格式内容进行校对和修正。

    校对的格式是：
    {data_template}

    以下是待校验的文本：{answer}，请你帮我校对和修正这个列表。
    '''

empty_template03='''
[]
'''

In [10]:
def filter_and_print_report(data_list, strength_key='关系强度', threshold=7.5):
    def is_valid_strength(strength):
        try:
            return float(strength) >= threshold
        except ValueError:
            return False

    original_count = len(data_list)
    filtered_list = [item for item in data_list if is_valid_strength(item[strength_key])]
    filtered_count = len(filtered_list)

    removed_count = original_count - filtered_count
    removed_percentage = (removed_count / original_count) * 100 if original_count > 0 else 0

    # 打印报告
    print(f"原始条目数: {original_count}")
    print(f"筛选后条目数: {filtered_count}")
    print(f"被筛掉的条目数: {removed_count}")
    print(f"被筛掉的百分比: {removed_percentage:.2f}%")

    return filtered_list




### * 将性质也设置为节点，并加入和数学对象的关系

In [11]:
def shift_properties(f1, f2):
    # 从f1中提取唯一的性质
    unique_list_of_properties = []
    for item in f1:
        if '性质' in item and item['性质']:
            for prop in item['性质']:
                if prop not in unique_list_of_properties:
                    unique_list_of_properties.append(prop)

    # 将性质也作为节点，添加到f1
    for prop in unique_list_of_properties:
        f1.append({'数学对象': prop, '定义': '一种性质'})

    # 向f2中添加对象及对应属性间的关系（此处设置为*）
    for item in f1:
        if '性质' in item and item['性质']:
            for prop in item['性质']:
                f2.append({
                    '出发节点': item['数学对象'],
                    '到达节点': prop,
                    '关系名称': '*',
                    '解释': f'{prop}是{item["数学对象"]}的性质',
                    '关系强度': '10'
                })

    # 从f1中移除'性质'键值对
    for item in f1:
        if '性质' in item:
            del item['性质']

    return f1, f2

In [12]:
def main_process(file_path, num_threads=500, checkpoint=100):
    global entity_pairs_explanation, extracted_entity_result

    # 初始化处理器
    entity_extractor = MultiProcessor(llm=llm, parse_method=parser.parse_dict, data_template=data_template01, 
                                      prompt_template=prompt_template01, correction_template=correction_prompt, 
                                      validator=validation01, back_up_llm=None)
    entity_explainer = MultiProcessor(llm=llm, parse_method=parser.parse_dict, data_template=data_template02, 
                                      prompt_template=prompt_template02, correction_template=correction_prompt, 
                                      validator=validation02, back_up_llm=None)
    left_relation_extractor = MultiProcessor(llm=llm, parse_method=parser.parse_dict, data_template=data_template03, 
                                             prompt_template=prompt_template03left, correction_template=correction_prompt, 
                                             validator=validation03)
    right_relation_extractor = MultiProcessor(llm=llm, parse_method=parser.parse_dict, data_template=data_template03, 
                                              prompt_template=prompt_template03right, correction_template=correction_prompt, 
                                              validator=validation03)

    # 处理文本
    text_list = divider.divide(file_path)
    text_dict = {index: {"pos1": value} for index, value in enumerate(text_list)}
    
    # 提取实体
    entity_dict = entity_extractor.multitask_perform(text_dict, num_threads=num_threads, checkpoint=checkpoint, 
                                                     Active_Reload=False, Active_Transform=False)
    filtered_data = filter_dict_of_lists(entity_dict)
    processed_data = process_dict_of_lists(filtered_data)
    entity_pairs = create_entity_pairs(processed_data)
    extracted_entities = extract_entities(processed_data)
    
    # 处理提取的实体
    processed_extracted_entities = replace_indices_with_text(extracted_entities, text_list)
    processed_entity_pairs = replace_indices_with_text(entity_pairs, text_list)
    
    # 解释实体
    extracted_entity_result = entity_explainer.multitask_perform(processed_extracted_entities, num_threads=num_threads, 
                                                                 checkpoint=checkpoint, Active_Transform=False)
    extracted_entity_result = list(extracted_entity_result.values())
    merged_result = merge_results(extracted_entity_result)
    
    # 提取关系
    left_entity_pairs_explanation = left_relation_extractor.multitask_perform(processed_entity_pairs, num_threads=num_threads, 
                                                                              checkpoint=checkpoint, Active_Transform=False)
    right_entity_pairs_explanation = right_relation_extractor.multitask_perform(processed_entity_pairs, num_threads=num_threads, 
                                                                                checkpoint=checkpoint, Active_Transform=False)
    left_entity_pairs_explanation = list(left_entity_pairs_explanation.values())
    right_entity_pairs_explanation = list(right_entity_pairs_explanation.values())
    entity_pairs_explanation = left_entity_pairs_explanation + right_entity_pairs_explanation
    
    # 过滤和报告
    filtered_entity_pairs = filter_and_print_report(entity_pairs_explanation)
    entity_pairs_explanation = filtered_entity_pairs
    
    # 移动属性
    extracted_entity_result, entity_pairs_explanation = shift_properties(extracted_entity_result, entity_pairs_explanation)

    return extracted_entity_result,entity_pairs_explanation


### 调用主函数

In [13]:
file_path = r'final_markdown.md'
f1,f2=main_process(file_path)
print('节点信息:\n',f1)
print('边信息：\n',f2)

  0%|          | 0/26 [00:00<?, ?it/s]

Error in task_perform: Error occurred during the API request: HTTPSConnectionPool(host='api.openai-next.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000272450C6D90>: Failed to resolve 'api.openai-next.com' ([Errno 11001] getaddrinfo failed)"))
An error occurred: Error occurred during the API request: HTTPSConnectionPool(host='api.openai-next.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000272450C6D90>: Failed to resolve 'api.openai-next.com' ([Errno 11001] getaddrinfo failed)")). Attempt 1/2
Error in task_perform: Error occurred during the API request: HTTPSConnectionPool(host='api.openai-next.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000272450C12D0>: Failed to resolve '

100%|██████████| 26/26 [00:41<00:00,  1.60s/it]


Final save_checkpoint call
Checkpoint updated and saved at checkpoint\checkpoint.json.
Final results - Total tasks: 26, Completed tasks: 26


  1%|          | 1/178 [00:04<12:19,  4.18s/it]

...

Error in task_perform: 解析失败，错误信息：invalid syntax (<unknown>, line 8)。原文字串为```json
{
    "出发节点": "存在定理",
    "到达节点": "凸函数",
    "关系名称": "特化关系",
    "关系解释": "设$S \\subset \\mathbb{R}^n$是非空的凸集.函数$f: S \\to \\mathbb{R}$称为是凸的,如果对于所有$(x, y) \\in S^2$和所有$t \\in [0, 1]$,有$f(t \\cdot x + (1-t) \\cdot y) \\leq t \\cdot f(x) + (1-t) \\cdot f(y)$",
    "关系强度": "10"
}
```

```json
{
    "出发节点": "存在定理",
    "到达节点": "严格凸函数",
    "关系名称": "特化关系",
    "关系解释": "在实向量空间(或仿射空间)$E$中,若函数$f: E \\rightarrow \\mathbb{R} \\cup \\{ +\\infty \\}$满足对所有不同的$x, y \\in E$及任意$t \\in (0, 1)$,有$f(t \\cdot x + (1-t) \\cdot y) < t \\cdot f(x) + (1-t) \\cdot f(y)$,则称$f$为严格凸函数",
    "关系强度": "10"
}
```

```json
{
    "出发节点": "存在定理",
    "到达节点": "强凸函数",
    "关系名称": "特化关系",
    "关系解释": "如果一个函数$f: E \\rightarrow \\mathbb{R} \\cup \\{ +\\infty \\}$对于所有不同的$x, y \\in E$及任意$t \\in (0, 1)$,满足$f(t \\cdot x + (1-t) \\cdot y) \\leq t \\cdot f(x) + (1-t) \\cdot f(y) - \\frac{m}{2}t(1-t)\\|x-y\\|^2$,其中$m > 0$,则称$f$为强凸函数,且说它是$m$-强凸的",
    "关系

100%|█████████▉| 1402/1403 [02:05<00:12, 12.14s/it]

Error in task_perform: Error occurred during the API request: 524 Server Error:  for url: https://api.openai-next.com/v1/chat/completions
An error occurred: Error occurred during the API request: 524 Server Error:  for url: https://api.openai-next.com/v1/chat/completions. Attempt 2/2


100%|██████████| 1403/1403 [02:12<00:00, 10.62it/s]

Error in task_perform: 解析失败，错误信息：invalid syntax (<unknown>, line 8)。原文字串为```json
{
    "出发节点": "凸集",
    "到达节点": "凸函数",
    "关系名称": "包含关系",
    "关系解释": "设$S \\subset \\mathbb{R}^n$是非空的凸集,函数$f: S \\to \\mathbb{R}$称为是凸的,如果对于所有$(x, y) \\in S^2$和所有$t \\in [0, 1]$,有$f(t \\cdot x + (1-t) \\cdot y) \\leq t \\cdot f(x) + (1-t) \\cdot f(y)$",
    "关系强度": 9
}
```

```json
{
    "出发节点": "严格凸函数",
    "到达节点": "凸函数",
    "关系名称": "特化关系",
    "关系解释": "在实向量空间(或仿射空间)$E$中,若函数$f: E \\rightarrow \\mathbb{R}\\cup\\{ +\\infty\\}$满足对所有不同的$x,y\\in E$及任意$t\\in(0,1)$,有$f(t\\cdot x + (1-t)\\cdot y) < t\\cdot f(x) + (1-t)\\cdot f(y)$,则称$f$为严格凸函数",
    "关系强度": 10
}
```

```json
{
    "出发节点": "强凸函数",
    "到达节点": "凸函数",
    "关系名称": "特化关系",
    "关系解释": "如果一个函数$f: E \\rightarrow \\mathbb{R}\\cup\\{ +\\infty\\}$对于所有不同的$x,y\\in E$及任意$t\\in(0,1)$,满足$f(t\\cdot x + (1-t)\\cdot y) \\leq t\\cdot f(x) + (1-t)\\cdot f(y) - \\frac{m}{2}t(1-t)\\|x-y\\|^2$,其中$m>0$,则称$f$为强凸函数",
    "关系强度": 10
}
```

```json
{
    "出发节点": "二次可微函数",


In [14]:
def count_keys(dict_list):
    key_count = {}
    
    for dictionary in dict_list:
        for key in dictionary.keys():
            if key in key_count:
                key_count[key] += 1
            else:
                key_count[key] = 1
    
    # 按出现次数从大到小排列
    sorted_key_count = dict(sorted(key_count.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_key_count
print(count_keys(f1))

{'数学对象': 251, '定义': 246, '应用场景': 137, '适用条件': 134, '计算方法': 1}


In [15]:
def count_relation_names(relations_list):
    relation_name_count = {}
    
    for relation_dict in relations_list:
        relation_name = relation_dict['关系名称']
        if relation_name in relation_name_count:
            relation_name_count[relation_name] += 1
        else:
            relation_name_count[relation_name] = 1
    
    # 按出现次数从大到小排列
    sorted_relation_name_count = dict(sorted(relation_name_count.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_relation_name_count

print(count_relation_names(f2))

{'包含关系': 949, '应用关系': 502, '特化关系': 247, '*': 193, '依赖关系': 75, '必要条件': 42, '必要条件关系': 29, '并列关系': 24, '支持关系': 16, '条件关系': 15, '收敛关系': 12, '辅助关系': 11, '工具关系': 10, '存在关系': 10, '存在性关系': 10, '近似关系': 9, '前提条件关系': 9, '导致关系': 9, '互补关系': 9, '表示关系': 7, '生成关系': 7, '决定关系': 7, '影响关系': 7, '相关关系': 6, '基础关系': 6, '前提关系': 5, '前提条件': 5, '促进关系': 5, '定义关系': 5, '必要关系': 5, '改进关系': 4, '扩展关系': 4, '解决方法': 3, '实现关系': 3, '优化关系': 3, '补集关系': 3, '同义关系': 3, '限制关系': 3, '基于关系': 3, '关联关系': 2, '替代关系': 2, '补充条件': 2, '满足关系': 2, '推导关系': 2, '优化方法的进化关系': 2, '优化方法的并列关系': 2, '蕴含关系': 2, '唯一性关系': 2, '构成关系': 2, '求解方法': 2, '对立关系': 2, '存在性保证': 2, '互斥关系': 2, '定义于关系': 1, '描述关系': 1, '组成部分关系': 1, '辅助工具关系': 1, '适用关系': 1, '补充关系': 1, '数学工具关系': 1, '排斥关系': 1, '存在性条件': 1, '保证关系': 1, '逻辑关系': 1, '保证极值存在关系': 1, '目标关系': 1, '确定关系': 1, '排除关系': 1, '求解关系': 1, '对应关系': 1, '特例关系': 1, '逼近关系': 1, '约束关系': 1, '几何关系': 1, '基础方法': 1, '适用条件': 1, '应用条件': 1, '相关性': 1, '数学结构关系': 1, '充分条件': 1, '协作关系': 1, '辅助工具': 1, '数学关系': 1}


In [16]:
f1

[{'数学对象': '拉格朗日条件'},
 {'数学对象': '集合', '定义': '一个由确定的对象组成的整体'},
 {'数学对象': '极方向', '定义': '在线性规划中,极方向是约束集的边界方向,表示在约束条件下变量变化的最大方向.'},
 {'数学对象': '可分离的', '定义': '在优化问题中,如果目标函数和约束条件可以分别处理,则称为可分离的.'},
 {'数学对象': '外部罚函数法', '定义': '一种优化方法,通过在目标函数中引入惩罚项来处理约束条件'},
 {'数学对象': '凸多面体',
  '定义': '由多个平面多边形围成的立体图形',
  '适用条件': '在优化问题中,约束条件形成的空间',
  '应用场景': '线性规划问题的可行解区域'},
 {'数学对象': '雅可比矩阵', '定义': '一个由多元函数的一阶偏导数组成的矩阵'},
 {'数学对象': '泰勒展开', '定义': '将一个函数表示为无穷级数的形式,由其在某点的导数决定.'},
 {'数学对象': '直线段',
  '定义': '在二维空间中,由两个端点确定的线段',
  '适用条件': '在给定两个端点的情况下',
  '应用场景': '几何学中的线段表示'},
 {'数学对象': '可行基底',
  '定义': '在优化问题中,满足所有约束条件的基底',
  '适用条件': '线性规划问题中,满足所有约束条件',
  '应用场景': '线性规划问题的求解过程中'},
 {'数学对象': '长方体盒子', '定义': '一个由六个矩形面组成的立体几何图形', '应用场景': '包装设计、建筑结构、数学教学'},
 {'数学对象': '下降方向', '定义': '在优化问题中,使得目标函数在该方向上减小的方向'},
 {'数学对象': '二次规划', '定义': '一种优化问题,目标函数是二次的,约束条件是线性的.'},
 {'数学对象': '最小化问题',
  '定义': '在约束条件下,寻找函数的最小值',
  '适用条件': '约束合格且函数在点处可微',
  '应用场景': '非线性约束优化问题'},
 {'数学对象': '二次型', '定义': '一个关于n个变量的二次齐次多项式', '应用场景': '优化问题、几何学、物理学'},
 {'数

In [17]:
f2

[{'出发节点': '矩阵',
  '到达节点': '集合',
  '关系名称': '包含关系',
  '关系解释': '矩阵可以看作是集合的一种特殊形式,其中元素按行和列排列.',
  '关系强度': '8'},
 {'出发节点': '拉格朗日函数',
  '到达节点': 'KKT条件',
  '关系名称': '包含关系',
  '关系解释': 'KKT条件是拉格朗日函数在最优化问题中的最优性必要条件',
  '关系强度': '10'},
 {'出发节点': '单位矩阵',
  '到达节点': '可行基底',
  '关系名称': '包含关系',
  '关系解释': '在某些线性规划问题中,单位矩阵可以直接作为初始的可行基底使用,因为单位矩阵的列向量是线性独立的,并且可以满足约束条件',
  '关系强度': '8'},
 {'出发节点': '凸多面体',
  '到达节点': '线性优化问题',
  '关系名称': '包含关系',
  '关系解释': '线性优化问题中的可行解集通常构成一个凸多面体,即在凸多面体内部寻找使目标函数最小的点',
  '关系强度': '9'},
 {'出发节点': '强凸函数',
  '到达节点': '海森矩阵',
  '关系名称': '包含关系',
  '关系解释': '强凸函数的定义要求其海森矩阵在所有点上都是正定的',
  '关系强度': '10'},
 {'出发节点': '牛顿法',
  '到达节点': '线性化',
  '关系名称': '包含关系',
  '关系解释': '牛顿法在每一步迭代中通过线性化目标函数来近似求解非线性方程或优化问题',
  '关系强度': '9'},
 {'出发节点': '凸多面体',
  '到达节点': '凸集',
  '关系名称': '特化关系',
  '关系解释': '凸多面体是凸集的一种特殊形式,它是通过有限数量的线性不等式定义的凸集.',
  '关系强度': '9'},
 {'出发节点': '迭代',
  '到达节点': '集合',
  '关系名称': '包含关系',
  '关系解释': '在数学优化中,迭代过程通常会生成一系列的解,这些解可以构成一个集合,表示在迭代过程中所有可能的解的集合.',
  '关系强度': '8'},
 {'出发节点': '可行基底',
  '到达节点': '矩阵',